Registro de los mejores makespans y número de iteraciones

## Best Move

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [58]:
# load data to dataframes
def load_data(data_directory, name):
    dfs = []

    for i in range(number_of_files):
        dfs.append([])
        file_df = pd.read_csv(data_directory + name.replace("N", str(i + 1)))
        column_names = file_df.columns.values.tolist()

        # remove trailing spances from column names
        column_names = [column_name.strip() for column_name in column_names]
        file_df.columns = column_names

        idx = [-1] + file_df[file_df['current_makespan'] == ' current_makespan'].index.tolist()

        for j in range(number_of_examples - 1):
            dfs[i].append(file_df.loc[idx[j] + 1:idx[j + 1] - 2,:])

        dfs[i].append(file_df.loc[idx[number_of_examples - 1] + 1:,:])
        
    return dfs

In [59]:
numeric_columns = ['best_makespan', 'current_makespan', 'current_temp', 'iter_number_in_temp',
                      'iter_per_temp', 'max_temperature', 'min_temperature', 'num_jobs',
                      'num_machines', 'num_operations', 'temp_change_ratio', 'worst_makespan', 'prob_acceptance']

string_columns = ['should_accept', 'move_direction', 'equal_count','first_op_moved', 'last_op_moved', 
                      'longer_count', 'operation_duration', 'selected_machine', 'shorter_count', 
                      'switch_operation_1', 'violates_constraint','moves']

def cast_types(dfs, numeric_columns, string_columns):
    # for each dataframe remove the first row from data
    for file_df in dfs:
        for df in file_df:
            df.drop(df.index[0], inplace=True)

    # for each dataframe, cast columns to appropriate types
    for file in range(number_of_files):
        for example in range(number_of_examples):
            for column in numeric_columns:
                if column in dfs[file][example].columns.values.tolist():
                    dfs[file][example].loc[:, column] = \
                            dfs[file][example].loc[:,column].apply(pd.to_numeric, errors='coerce')

            for column in string_columns:
                if column in dfs[file][example].columns.values.tolist():
                    dfs[file][example].loc[:, column] = \
                            dfs[file][example].loc[:,column].astype(str)

In [68]:
def highlight_min(s):
    '''
    highlight the minimum in a Series yellow.
    '''
    is_min = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_min]

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [65]:
heuristics = ["BEST", "BEST2", "FIRST", "SEQ", "DE", "PS"]
files = ["JSSP_3_fN.csv", "JSSP_4_fN.csv", "JSSP_5_fN.csv", "JSSP_6_fN.csv", "JSSP_7_fN.csv", "JSSP_2_fN.csv"]
directory = "../../data/first_run/"
number_of_files = 7
number_of_examples = 20
all_dfs = {}
makespans_dfs = {}
iterations_dfs = {}

for heuristic, file in zip(heuristics, files):
    all_dfs[heuristic] = load_data(directory, file)
    cast_types(all_dfs[heuristic], numeric_columns, string_columns)

    makespans_dfs[heuristic] = pd.DataFrame()
    iterations_dfs[heuristic] = pd.DataFrame()
    for file in range(number_of_files):
        values = []
        iters = []
        for example in range(10):
            values.append(all_dfs[heuristic][file][example].loc[:, "best_makespan"].min())
            iters.append(all_dfs[heuristic][file][example].shape[0])
            
        column_name = "{0}x{1}:".format(int(all_dfs[heuristic][file][0].loc[0:1, "num_jobs"]), 
                                        int(all_dfs[heuristic][file][0].loc[0:1, "num_machines"]))
        
        makespans_dfs[heuristic][column_name] = pd.Series(values)
        iterations_dfs[heuristic][column_name] = pd.Series(iters)

In [67]:
from IPython.display import display

for heuristic in heuristics:
    print(heuristic)
    display(makespans_dfs[heuristic].style.apply(highlight_min))

BEST


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,2031,2736,3412,3601,4256,5141,6417
1,2305,2640,3140,3761,4412,5097,6091
2,2465,2820,3323,3489,4500,4931,6230
3,2061,2586,2832,3235,4187,5672,6160
4,2497,2509,3152,3769,4233,5305,6409
5,2260,2662,2904,3305,4184,5231,6192
6,2262,2618,3466,3841,4801,5328,5846
7,2320,2653,3273,3708,4526,5193,5956
8,2120,2655,2780,3300,4398,5361,6338
9,2369,2720,3334,3847,3897,4712,6303


BEST2


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,2073,2567,2946,3213,4031,5192,5474
1,2114,2395,2745,3581,4232,5019,6021
2,1994,2467,3552,3032,3697,5060,6387
3,2161,2618,2945,3486,3819,5369,6121
4,2140,2531,2977,3455,4059,4982,5620
5,2241,2728,2940,3390,4032,4809,6039
6,2061,2545,3045,3157,4075,5117,6144
7,2221,2667,3178,3209,3727,4938,5754
8,2284,2810,3613,3353,4130,5503,6014
9,2113,2702,2868,3687,4930,5187,6087


FIRST


...

,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,2294,2277,2847,3382,4133,5052,6002
1,2209,2223,3068,3500,4249,5020,5411
2,2135,2311,3057,3416,3866,5083,6033
3,2382,2487,2675,3060,3883,4723,5845
4,2041,2323,2902,3098,4082,5029,5687
5,1920,2556,2718,3340,4108,4904,5039
6,1966,2473,2742,3345,3870,4925,5583
7,2129,2684,2923,3475,4144,5283,5556
8,2182,2312,2927,3582,4196,5338,5525
9,2016,2561,3009,2987,3757,4918,5450


DE


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,2130,2785,3065,3432,4340,5048,5680
1,2221,2498,3236,2810,4011,4626,5241
2,2386,2547,2863,3207,4221,5136,5359
3,2118,2362,2939,3440,3830,4984,5171
4,2083,2477,3001,3294,4138,5356,5903
5,2192,2487,3052,3175,4243,4701,5496
6,2076,2810,3078,3310,4368,4751,5641
7,2151,2138,3365,3106,4288,5200,5999
8,2173,2460,2912,3489,3921,4567,5991
9,2362,2465,3283,3479,3756,4852,5779


PS


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,2210,2678,3070,3258,4528,5306,5687
1,2143,2640,3155,3424,4291,5459,5908
2,2172,2406,3185,3573,3866,4755,5796
3,2366,2690,3378,3276,4413,5137,6386
4,2459,2406,3625,3628,4389,5158,5466
5,2199,2761,3244,3299,4204,5220,5719
6,2216,2379,3261,3344,4478,5232,6100
7,2504,2492,3415,3221,4034,4861,5972
8,2301,2234,3139,3406,4616,5221,5721
9,2419,2665,3308,3199,4037,5336,5774


In [69]:
for heuristic in heuristics:
    print(heuristic)
    display(iterations_dfs[heuristic].style.apply(highlight_max))

BEST


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,271,271,271,271,271,271,271
1,271,271,271,271,271,271,271
2,271,271,271,271,271,271,271
3,271,271,271,271,271,271,271
4,271,271,271,271,271,271,271
5,271,271,271,271,271,271,271
6,271,271,271,271,271,271,271
7,271,271,271,271,271,271,271
8,271,271,271,271,271,271,271
9,271,271,271,271,271,271,271


BEST2


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,290,285,281,291,291,296,290
1,284,295,285,280,289,285,282
2,289,295,282,285,283,281,284
3,291,293,287,292,282,285,285
4,286,285,289,289,284,285,288
5,280,293,284,287,284,285,286
6,282,288,283,290,288,287,277
7,294,284,284,294,286,288,284
8,288,290,284,284,283,286,287
9,289,289,280,283,291,285,281


FIRST


...

,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,74,58,60,43,40,30,31
1,74,61,56,41,48,32,30
2,68,59,62,53,43,24,30
3,71,58,58,46,41,32,31
4,79,64,65,45,43,30,28
5,76,54,64,45,41,28,32
6,70,57,54,46,46,30,30
7,73,60,52,41,38,33,32
8,71,62,60,46,39,32,34
9,82,56,54,39,41,30,32


DE


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,799,799,799,799,799,799,799
1,799,799,799,799,799,799,799
2,799,799,799,799,799,799,799
3,799,799,799,799,799,799,799
4,799,799,799,799,799,799,799
5,799,799,799,799,799,799,799
6,799,799,799,799,799,799,799
7,799,799,799,799,799,799,799
8,799,799,799,799,799,799,799
9,799,799,799,799,799,799,799


PS


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,799,799,799,799,799,799,799
1,799,799,799,799,799,799,799
2,799,799,799,799,799,799,799
3,799,799,799,799,799,799,799
4,799,799,799,799,799,799,799
5,799,799,799,799,799,799,799
6,799,799,799,799,799,799,799
7,799,799,799,799,799,799,799
8,799,799,799,799,799,799,799
9,799,799,799,799,799,799,799


## Algoritmos basados en intuición

Con las observaciones realizadas y las conclusiones derivadas a partir de ellas, se determinó la existencia de mejora considerable al realizar una hiperheuristica que tome las ventajas de las heuristicas utilizadas solamente. Por ello, se esarrollaron los siguientes algoritmos:

### PS Exchange with ascending sequential search interleaved with best move algorithm

In general, PS Exchange will be given 100 iterations without affecting temperature at the beginning, its main function is to find a suitable initial solution for the rest of the heuristics. The reson for this selection is based on the rapid decrease in makespan done by the heuristic. This can be seen in the following graphs:

![ps_1](pictures/ps_1.png)

Once a candidate solution is obtained, best move will start being used for 70 iterations. Sequential search will begin excution under two scenarios.

1. If 10 iterations pass with best move and no improvement is obtained, change best move for sequential search.
2. Best Move's 70 iterations pass, at which sequential search will begin and do 5 machine selections. This is equivalent to $5n$ iterations in the worst case, where n is the number of operations in the machine.

The algorithm for now will run for some ~800 iterations, ~900 if we count the candidate selection. Therefore, in average it is expected that the heuristic distribution will be:

* 0   - 100: PS Exchange
* 100 - 170: Best Move
* 170 - 240: Sequential Search
* 240 - 310: Best Move
* 310 - 400: Sequential Search
* 400 - 450: Best Move
* 450 - 510: Sequential Search
* ... 